In [6]:
import os
import pandas as pd
from glob import glob
import cv2
import json

ANNO_DIR = 'C:/Users/owner/PycharmProjects/Counting_in_Agri/Data/Melons/detection annotations'
IMAGE_DIR = 'C:/Users/owner/PycharmProjects/Counting_in_Agri/Data/Melons/images'
save_annotations_path = 'C:/Users/owner/PycharmProjects/Counting_in_Agri/Data/Melons/coco/annotations'

In [7]:
def xyxy2xywh(df_row):
    x = df_row[0]
    y = df_row[1]
    w = df_row[2]-df_row[0]
    h = df_row[3]-df_row[1]
    return x, y, w, h

In [9]:
sets = {'train', 'val', 'test'}
categories = [{'supercategory': None, 'id': 1, 'name': 'Melons'}]
for current_set in sets:
    image_id = 0
    images = []
    annotations_id = 0
    annotations = []
    json_file = os.path.join(save_annotations_path, 'instances_' + current_set + '.json') 
    set_anno_dir = os.path.join(ANNO_DIR, current_set)
    set_image_dir = os.path.join(IMAGE_DIR, current_set)
    for csv_file in glob(os.path.join(set_anno_dir, '*csv')):
        image_name = csv_file.split('\\')[-1].split('.')[0] + '.JPG'
        csv_annoations = pd.read_csv(csv_file)
        image = cv2.imread(os.path.join(set_image_dir, image_name))
        image_h, image_w, _ = image.shape
        images.append({
            'file_name': image_name, 'height': image_h, 'width': image_w,
            'id': image_id
        })
        image_anno = []
        for i, row in csv_annoations.iterrows():
            x,y,w,h = xyxy2xywh(row)
            ann = {'area': w*h, 'iscrowd': 0, 'image_id':
                    image_id, 'bbox': [x, y, w, h],
                    'category_id': 1, 'id': annotations_id, 'ignore': 0}
            annotations.append(ann)
            annotations_id += 1
        image_id += 1
    json_dict = {
        'categories': categories,
        'images': images,
        'annotations': annotations
    }
    json_fp = open(json_file, 'w')
    json_str = json.dumps(json_dict)
    json_fp.write(json_str)
